In [40]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd
import numpy as np

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "80%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [41]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "job_spaces_2019.gdb"]
gdb = os.path.join(outputs[0], outputs[1])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

In [42]:
parcels = r"E:\Projects\REMM-Input-Data-Prep-2019\Parcels\2020-WFRC\Outputs\recompile3.gdb\parcels_2019"
dws_pts = r'E:\Projects\REMM-Input-Data-Prep-2019\Job-Space-Distribution\Inputs\RegionDWS_REMM.gdb\RegionDWS_Employment'
taz = r'E:\Data\Boundaries\TAZ 2021-06-12\USTM TAZ - 2021-06-12.shp' # filter REMM == 1
taz_lyr = arcpy.MakeFeatureLayer_management(taz, 'taz_pts', '''SUBAREAID = 1''')

In [43]:
# # spatial join
# target_features = parcels
# join_features = dws_pts
# output_features = os.path.join(gdb, "parcel_dws_sj")

# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(target_features)
# fieldmappings.addTable(join_features)

# # total market value
# fieldindex = fieldmappings.findFieldMapIndex('Emp2019')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # run the spatial join, use 'Join_Count' for number of units
# sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

In [44]:
# # spatial join
# target_features = sj
# join_features = taz_lyr
# output_features = os.path.join(gdb, "parcel_dws_taz_sj")

# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(target_features)
# fieldmappings.addTable(join_features)

# # run the spatial join, use 'Join_Count' for number of units
# sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "HAVE_THEIR_CENTER_IN")
# parcels_sdf = pd.DataFrame.spatial.from_featureclass(sj2[0])

In [45]:
# print(list(parcels_sdf.columns))

In [46]:
# # remove extra fields and export feature class
# parcels_sdf = parcels_sdf[['CO_NAME', 'COUNTY_ID', 'PARCEL_ID', 'TOTAL_MKT_VALUE', 'land_value', 'residential_units', 'building_sqft', 'FLOORS_CNT', 'year_built', 
#                           'IS_OUG', 'max_dua', 'max_far', 'max_height', 'type1', 'type2', 'type3', 'type4', 'type5', 'type6', 'type7', 'type8', 'agriculture', 'basebldg', 
#                           'NoBuild', 'redev_friction', 'building_type_id', 'x', 'y', 'parcel_id_REMM', 'parcel_acres', 'Split', 'Split_Factor', 'TYPE','Emp2019', 'TAZID', 
                        #   'SA_TAZID', 'CO_TAZID','SHAPE']].copy()

# parcels_sdf.spatial.to_featureclass(location=os.path.join(gdb, "_00_base_year_parcels_with_total_jobs"),sanitize_columns=False)

In [47]:
# # export to csv for pivoting
# filtered_parcels = parcels_sdf[(parcels_sdf['building_sqft'] > 100) & (parcels_sdf['Emp2019'] > 0)].copy()
# del filtered_parcels['SHAPE']
# filtered_parcels.to_csv(os.path.join(outputs[0], 'base_year_parcels_with_total_jobs.csv'), index=False)

In [48]:
# # subset parcels by county, then filter for those that have jobs and square footage
# d = parcels_sdf[(parcels_sdf['COUNTY_ID']== '11') & (parcels_sdf['building_sqft'] > 100) & (parcels_sdf['Emp2019'] > 0)].copy()
# w = parcels_sdf[(parcels_sdf['COUNTY_ID']== '57') & (parcels_sdf['building_sqft'] > 100) & (parcels_sdf['Emp2019'] > 0)].copy()
# sl = parcels_sdf[(parcels_sdf['COUNTY_ID']== '35') & (parcels_sdf['building_sqft'] > 100) & (parcels_sdf['Emp2019'] > 0)].copy()

## Davis County


In [49]:
# print('--FREQ--')
# print(d['TYPE'].value_counts())

In [50]:
# print('--MIN--')
# print(d.groupby('TYPE')['building_sqft'].min())

In [51]:
# print('--MAX--')
# print(d.groupby('TYPE')['building_sqft'].max())

## Weber County

In [52]:
# print('--FREQ--')
# print(w['TYPE'].value_counts())

In [53]:
# print('--MIN--')
# print(w.groupby('TYPE')['building_sqft'].min())

In [54]:
# print('--MAX--')
# print(w.groupby('TYPE')['building_sqft'].max())

## Salt Lake County

In [55]:
# print('--FREQ--')
# print(sl['TYPE'].value_counts())

In [56]:
# print('--MIN--')
# print(sl.groupby('TYPE')['building_sqft'].min())

In [57]:
# print('--MAX--')
# print(sl.groupby('TYPE')['building_sqft'].max())

# Sum Jobs by TAZ

In [58]:
dws_pts_sdf = pd.DataFrame.spatial.from_featureclass(dws_pts)
ind_job_pts = dws_pts_sdf[(dws_pts_sdf['REMMSectorID'].isin([5,10]))].copy()
ind_job_pts.spatial.to_featureclass(location=os.path.join(gdb, "_01_industrial_job_pts"),sanitize_columns=False)
non_ind_job_pts = dws_pts_sdf[~(dws_pts_sdf['REMMSectorID'].isin([5,10]))].copy()
non_ind_job_pts.spatial.to_featureclass(location=os.path.join(gdb, "_01_non_industrial_job_pts"),sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Job-Space-Distribution\\Outputs\\job_spaces_2019.gdb\\_01_non_industrial_job_pts'

In [59]:
# spatial join
target_features = taz_lyr
join_features = os.path.join(gdb, "_01_industrial_job_pts")
output_features = os.path.join(gdb, "_01_taz_ind_jobs_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('Emp2019')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

In [60]:
# spatial join
target_features = taz_lyr
join_features = os.path.join(gdb, "_01_non_industrial_job_pts")
output_features = os.path.join(gdb, "_01_taz_non_ind_jobs_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('Emp2019')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

# Sum Building Square Footage by TAZ

In [61]:
# convert to pts
parcels_pts = arcpy.FeatureToPoint_management(parcels, os.path.join(gdb, "_02_parcels_pts"), "INSIDE")
parcels_pts_sdf = pd.DataFrame.spatial.from_featureclass(parcels_pts[0])

# split into industrial and > 100 sqft
industrial = parcels_pts_sdf[(parcels_pts_sdf['building_type_id']==3) & (parcels_pts_sdf['building_sqft'] > 100)].copy()
industrial.spatial.to_featureclass(location=os.path.join(gdb, "_02_industrial_parcels"),sanitize_columns=False)

# get non-industrial and > 100 sqft
non_industrial = parcels_pts_sdf[(parcels_pts_sdf['building_type_id'].isin([4,5,6,7,9,13])) & (parcels_pts_sdf['building_sqft'] > 100)].copy()
non_industrial.spatial.to_featureclass(location=os.path.join(gdb, "_02_non_industrial_parcels"),sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Job-Space-Distribution\\Outputs\\job_spaces_2019.gdb\\_02_non_industrial_parcels'

In [62]:
# spatial join -industrial
target_features = taz_lyr
join_features = os.path.join(gdb, "_02_industrial_parcels")
output_features = os.path.join(gdb, "_02_taz_ind_sj_sum")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('building_sqft')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
sj3 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

In [63]:
# spatial join - non industrial
target_features = taz_lyr
join_features = os.path.join(gdb, "_02_non_industrial_parcels")
output_features = os.path.join(gdb, "_02_taz_nonind_sj_sum")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('building_sqft')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
sj4 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

# Combine into one table

In [64]:
# industrial
sj_industrial_jobs_sum = pd.DataFrame.spatial.from_featureclass(sj[0])[['CO_TAZID','Emp2019']]
sj_industrial_jobs_sum.columns = ['CO_TAZID', 'IND_JOBS_SUM']

sj_industrial_sqft_sum = pd.DataFrame.spatial.from_featureclass(sj3[0])[['CO_TAZID','building_sqft']]
sj_industrial_sqft_sum.columns = ['CO_TAZID', 'IND_SQFT_SUM']

# non industrial
sj_non_industrial_jobs_sum = pd.DataFrame.spatial.from_featureclass(sj2[0])[['CO_TAZID','Emp2019']]
sj_non_industrial_jobs_sum.columns =  ['CO_TAZID', 'NON_IND_JOBS_SUM']

sj_non_industrial_sqft_sum = pd.DataFrame.spatial.from_featureclass(sj4[0])[['CO_TAZID','building_sqft']]
sj_non_industrial_sqft_sum.columns =  ['CO_TAZID', 'NON_IND_SQFT_SUM']

In [65]:
# load taz
taz_sdf = pd.DataFrame.spatial.from_featureclass(taz)
taz_sdf = taz_sdf[taz_sdf['SUBAREAID'] == 1].copy()
taz_sdf = taz_sdf[['TAZID',	'SA_TAZID', 'CO_TAZID', 'CO_FIPS', 'SHAPE']].copy()

# join tables
taz_merge = taz_sdf.merge(sj_industrial_jobs_sum, left_on='CO_TAZID', right_on='CO_TAZID',how ='left')
taz_merge = taz_merge.merge(sj_industrial_sqft_sum, left_on='CO_TAZID', right_on='CO_TAZID',how ='left')
taz_merge = taz_merge.merge(sj_non_industrial_jobs_sum, left_on='CO_TAZID', right_on='CO_TAZID',how ='left')
taz_merge = taz_merge.merge(sj_non_industrial_sqft_sum, left_on='CO_TAZID', right_on='CO_TAZID',how ='left')

# Further Calculations & Export

In [66]:
# sqft per job assumptions
industrial_min_max = [500,1500] #400?
retail_min_max = [200,500]
office_min_max = [300,800]
gov_edu_min_max = [200,500]
mixed_min_max = [300,800]
other_min_max = [200,800]
non_industrial_min_max = [200,800] # took min of mins, and max of maxs

# calc min and max job spaces allowed
taz_merge['IND_MIN_JOB_SPACES'] = taz_merge['IND_SQFT_SUM'] / industrial_min_max[0]
taz_merge['IND_MAX_JOB_SPACES'] = taz_merge['IND_SQFT_SUM'] / industrial_min_max[1]

taz_merge['NON_IND_MIN_JOB_SPACES'] = taz_merge['NON_IND_SQFT_SUM'] / non_industrial_min_max[0]
taz_merge['NON_IND_MAX_JOB_SPACES'] = taz_merge['NON_IND_SQFT_SUM'] / non_industrial_min_max[1]

# calc job-job spaces difference
taz_merge['IND_MIN_SPACES_MINUS_JOBS'] =  taz_merge['IND_MIN_JOB_SPACES'] - taz_merge['IND_JOBS_SUM']
taz_merge['IND_MAX_SPACES_MINUS_JOBS'] =  taz_merge['IND_MAX_JOB_SPACES'] - taz_merge['IND_JOBS_SUM']

taz_merge['NON_IND_MIN_SPACES_MINUS_JOBS'] =   taz_merge['NON_IND_MIN_JOB_SPACES'] - taz_merge['NON_IND_JOBS_SUM']
taz_merge['NON_IND_MAX_SPACES_MINUS_JOBS'] = taz_merge['NON_IND_MAX_JOB_SPACES'] - taz_merge['NON_IND_JOBS_SUM']  

In [67]:
# export
taz_merge.spatial.to_featureclass(location=os.path.join(gdb, "_03_taz_job_spaces_summary"),sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Job-Space-Distribution\\Outputs\\job_spaces_2019.gdb\\_03_taz_job_spaces_summary'

In [68]:
# # load the taz
# SUBAREAID = 1
# taz_sdf = pd.DataFrame.spatial.from_featureclass(taz)
# taz_sdf = taz_sdf[taz_sdf['SUBAREAID'] == 1]
# taz_sdf = taz_sdf[['TAZID',	'SA_TAZID', 'CO_TAZID', 'CO_FIPS', 'SHAPE']].copy()

In [69]:
# # pivot the parcel np.min, np.max, np.mean
# pivot = filtered_parcels.pivot_table(index='CO_TAZID',columns='TYPE',values='BLDG_SQFT', aggfunc=np.min, fill_value=0).reset_index()
# pivot = pivot.astype(int)

In [70]:
# # merge the tables together
# taz_summary = taz_sdf.merge(pivot, left_on='CO_TAZID',right_on='CO_TAZID', how='left')
# taz_summary.head()

In [71]:
# parcels_df = pd.DataFrame.spatial.from_featureclass(parcels)
# parcels.columns

In [72]:
# d_parcels = parcels_df[(parcels_df['COUNTY_ID']== '11') & (parcels_df['BLDG_SQFT'] > 0)].copy()
# sl_parcels = parcels_df[(parcels_df['COUNTY_ID']== '35') & (parcels_df['BLDG_SQFT'] > 0)].copy()
# w_parcels = parcels_df[(parcels_df['COUNTY_ID']== '57') & (parcels_df['BLDG_SQFT'] > 0)].copy()
# parcels_gt = parcels_df[parcels_df['BLDG_SQFT'] > 0].copy()

In [73]:
# d_sqft_max = d_parcels.groupby('TYPE')['BLDG_SQFT'].max()
# sl_sqft_max = sl_parcels.groupby('TYPE')['BLDG_SQFT'].max()
# w_sqft_max = w_parcels.groupby('TYPE')['BLDG_SQFT'].max()

# d_sqft_min = d_parcels.groupby('TYPE')['BLDG_SQFT'].min()
# sl_sqft_min = sl_parcels.groupby('TYPE')['BLDG_SQFT'].min()
# w_sqft_min = w_parcels.groupby('TYPE')['BLDG_SQFT'].min()

# d_sqft_mean = d_parcels.groupby('TYPE')['BLDG_SQFT'].mean()
# sl_sqft_mean = sl_parcels.groupby('TYPE')['BLDG_SQFT'].mean()
# w_sqft_mean = w_parcels.groupby('TYPE')['BLDG_SQFT'].mean()

# d_sqft_sum = d_parcels.groupby('TYPE')['BLDG_SQFT'].sum()
# sl_sqft_sum = sl_parcels.groupby('TYPE')['BLDG_SQFT'].sum()
# w_sqft_sum = w_parcels.groupby('TYPE')['BLDG_SQFT'].sum()

In [74]:
# dws_pts_df = pd.DataFrame.spatial.from_featureclass(dws_pts)
# dws_pts_df.columns

In [75]:
# # split jobs into industrial and non-industrial
# ind_pts = dws_pts_df[dws_pts_df['REMMSectorID'].isin([5, 10])].copy()
# non_ind_pts = dws_pts_df[~(dws_pts_df['REMMSectorID'].isin([5, 10]))].copy()

# ind_pts.spatial.to_featureclass(location=os.path.join(gdb, 'dws_2019_industrial'),sanitize_columns=False)
# non_ind_pts.spatial.to_featureclass(location=os.path.join(gdb, 'dws_2019_nonindustrial'),sanitize_columns=False)

In [76]:
# # get pts that do not overlap with a remm parcel
# ind_pts_lyr = arcpy.MakeFeatureLayer_management(os.path.join(gdb, 'dws_2019_industrial'), 'ind_pts')
# non_ind_pts_lyr = arcpy.MakeFeatureLayer_management(os.path.join(gdb, 'dws_2019_nonindustrial'), 'non_ind_pts')  


# arcpy.SelectLayerByLocation_management(in_layer=ind_pts_lyr, overlap_type="INTERSECT",
#                                        select_features=parcels,
#                                        selection_type='NEW_SELECTION',
#                                        invert_spatial_relationship='INVERT')

# ind_pts_no_parcel = arcpy.conversion.FeatureClassToFeatureClass(in_features=ind_pts_lyr, 
#                                                                out_path=gdb, 
#                                                                out_name="ind_pts_no_parcel")

# arcpy.SelectLayerByLocation_management(in_layer=non_ind_pts_lyr, overlap_type="INTERSECT",
#                                        select_features=parcels,
#                                        selection_type='NEW_SELECTION',
#                                        invert_spatial_relationship='INVERT')


# non_ind_pts_no_parcel = arcpy.conversion.FeatureClassToFeatureClass(in_features=non_ind_pts_lyr, 
#                                                                out_path=gdb, 
#                                                                out_name="non_ind_pts_no_parcel")                                                                                     


In [77]:
# # use spatial join to summarize TOTAL_MKT_VALUE, LAND_MKT_VALUE, BLDG_SQFT
# target_features = parcels
# join_features = os.path.join(gdb, 'dws_2019_industrial')
# output_features = os.path.join(gdb, "parcel_dws_ind_sj")

# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(target_features)
# fieldmappings.addTable(join_features)

# # total market value
# fieldindex = fieldmappings.findFieldMapIndex('Emp2019')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # run the spatial join, use 'Join_Count' for number of units
# sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")

In [78]:
# # use spatial join to summarize TOTAL_MKT_VALUE, LAND_MKT_VALUE, BLDG_SQFT
# target_features = parcels
# join_features = os.path.join(gdb, 'dws_2019_nonindustrial')
# output_features = os.path.join(gdb, "parcel_dws_non_ind_sj")

# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(target_features)
# fieldmappings.addTable(join_features)

# # total market value
# fieldindex = fieldmappings.findFieldMapIndex('Emp2019')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # run the spatial join, use 'Join_Count' for number of units
# sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, "INTERSECT")